In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

In [2]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)
print("done!")

done!


In [3]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)

In [4]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    
def get_model():
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16,3,3, activation=p_activation) (input_1)
    img_1 = Conv2D(16,3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
    img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
    img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(128, 3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, 3,3, activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_3 = BatchNormalization(momentum=bn_model)(input_2)
    
    img_concat =  (Concatenate()([img_1, img_2, img_3]))
    
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
model = get_model()
model.summary()

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
X_1 (InputLayer)                 (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 73, 73, 16)    448         X_1[0][0]                        
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 71, 71, 16)    2320        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 35, 35, 16)    0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [21]:
model.load_weights(filepath="/mnt/extDisk/courses/data/statoil/data/keras_baseline/.model_weights.hdf5")

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
600/602 [============================>.] - ETA: 0s[0.23146934386503656, 0.88372094211388263]
####################
watch list evaluate:
200/200 [==============================] - 0s
[0.25437977910041809, 0.89500004053115845]


In [6]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8424/8424 [==============================] - 1s     


In [14]:
X_mix = np.concatenate([X_train,X_test])
X_angle_mix = np.concatenate([X_angle_train,X_angle_test])
y_mix = np.concatenate([y_train,np.squeeze(prediction)])

In [22]:
layers = model.layers
# Get the index of the first dense layer...
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable=True

In [24]:
file_path = "/mnt/extDisk/courses/data/statoil/data/pseudo_labeling/.model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model.fit([X_mix, X_angle_mix], y_mix, epochs=25
          , validation_data=([X_valid, X_angle_valid], y_valid)
         , batch_size=32
         , callbacks=callbacks)

Train on 9026 samples, validate on 200 samples
Epoch 1/25
9026/9026 [==============================] - 6s - loss: 0.3435 - acc: 0.0558 - val_loss: 5.3180 - val_acc: 0.6150
Epoch 2/25
9026/9026 [==============================] - 6s - loss: 0.3070 - acc: 0.0559 - val_loss: 3.0715 - val_acc: 0.7400
Epoch 3/25
9026/9026 [==============================] - 6s - loss: 0.3003 - acc: 0.0562 - val_loss: 2.9842 - val_acc: 0.7500
Epoch 4/25
9026/9026 [==============================] - 6s - loss: 0.2938 - acc: 0.0579 - val_loss: 1.4088 - val_acc: 0.7750
Epoch 5/25
9026/9026 [==============================] - 6s - loss: 0.2913 - acc: 0.0576 - val_loss: 0.8115 - val_acc: 0.8650
Epoch 6/25
9026/9026 [==============================] - 6s - loss: 0.2941 - acc: 0.0572 - val_loss: 5.6192 - val_acc: 0.6300
Epoch 7/25
9026/9026 [==============================] - 6s - loss: 0.2920 - acc: 0.0574 - val_loss: 1.5263 - val_acc: 0.8600
Epoch 8/25
9026/9026 [==============================] - 6s - loss: 0.2884 - ac

In [25]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate([X_mix, X_angle_mix], y_mix, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
9000/9026 [============================>.] - ETA: 0s[0.31303737000697418, 0.057832927622830502]
####################
watch list evaluate:
200/200 [==============================] - 0s
[0.26405501365661621, 0.87999999523162842]


In [26]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8200/8424 [============================>.] - ETA: 0s

In [27]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,4.767923e-01
1,4023181e,1.980759e-01
2,b20200e4,8.555650e-10
3,e7f018bb,9.840789e-01
4,4371c8c3,2.749164e-01
5,a8d9b1fd,3.614210e-01
6,29e7727e,5.286715e-01
7,92a51ffb,9.912357e-01
8,c769ac97,5.653215e-04
9,aee0547d,5.465093e-07


In [28]:
submission.to_csv("/mnt/extDisk/courses/data/statoil/data/train/pseudo_labeling.csv", index=False)